In [2]:
!nvidia-smi

Sat Nov 15 07:34:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [4]:
# Load Model and tokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map = "cuda", 
    torch_dtype = "auto", 
    trust_remote_code = True, 
)

# Creata a pipeline

generator = pipeline(
    "text-generation", 
    model = model, 
    tokenizer = tokenizer,
    return_full_text = False, 
    max_new_tokens = 50, 
    do_sample = False
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [7]:
prompt = "Write an email apologizing to sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt, use_cache = False)

print(output[0]["generated_text"])



Dear Sarah,

I am deeply sorry for the unfortunate incident that occurred in my garden. I understand that it must have been quite distressing for you to witness the damage caused to your beloved plants.




In [15]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLUActivation()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_fea

In [21]:
prompt = 'The Capital of france is'

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Tokenize the input prompt:
input_ids = input_ids.to("cuda")

# Get the output of the model before the lm_head
model_output = model.model(input_ids, use_cache=False)

# Get the output of the lm head
lm_head_output = model.lm_head(model_output[0])

In [22]:
model_output

BaseModelOutputWithPast(last_hidden_state=tensor([[[-0.3047,  1.1953,  0.2988,  ..., -0.3008,  0.6758,  0.1406],
         [-0.6367,  1.0234, -0.4805,  ...,  0.8125, -0.2461, -0.3164],
         [-0.3594,  1.1250,  1.5156,  ...,  0.0854,  0.1240,  0.3203],
         [-0.0530,  0.7188,  0.2617,  ...,  0.8359, -0.6641,  0.7266],
         [-0.6641,  1.1875,  0.8750,  ...,  0.0488,  0.1855,  0.0854],
         [-1.1016,  0.5742,  0.0913,  ...,  0.3906, -0.0396,  0.2715]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<MulBackward0>), past_key_values=None, hidden_states=None, attentions=None)

In [23]:
lm_head_output

tensor([[[24.7500, 24.8750, 22.7500,  ..., 19.0000, 19.0000, 19.0000],
         [26.6250, 28.6250, 26.0000,  ..., 21.7500, 21.7500, 21.7500],
         [33.0000, 31.5000, 32.5000,  ..., 26.6250, 26.6250, 26.6250],
         [14.6250, 12.6875, 15.1250,  ..., 10.0625, 10.0625, 10.0625],
         [32.2500, 31.8750, 34.0000,  ..., 27.5000, 27.5000, 27.5000],
         [27.3750, 27.8750, 26.2500,  ..., 20.7500, 20.7500, 20.7500]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)

In [25]:
token_id = lm_head_output[0, -1].argmax(-1)
tokenizer.decode(token_id)

'par'